In [ ]:
#default_exp web.blog

# Web Blog

> Handles journal UI (endpoint and view) operatations.

# TODO

- add journal download option

In [ ]:
#export
from flask import Blueprint
from flask import flash
from flask import g
from flask import redirect
from flask import render_template
from flask import request
from flask import url_for
from flask import send_from_directory
from werkzeug.exceptions import abort
from werkzeug.utils import secure_filename
import tempfile,time
import markdown
import markdown.extensions.fenced_code
from pathlib import Path

from web_journal.web.auth import login_required

bp = Blueprint("blog", __name__)

In [ ]:
#export
@bp.route("/")
@login_required
def index():
    """Show all the posts, most recent first."""
    posts = []
    if g.user is not None:
        posts = g.service.read_posts_by_author_id(g.user["id"])
    for post in posts: # TODO: move this to blog index template
        post['body']=markdown.markdown(post['body'], extensions=['fenced_code'])
    return render_template("blog/index.html", posts=posts)

In [ ]:
#export
def get_post(id, check_author=True):
    """Get a post and its author by id.
    Checks that the id exists and optionally that the current user is
    the author.
    :param id: id of post to get
    :param check_author: require the current user to be the author
    :return: the post with author information
    :raise 404: if a post with the given id doesn't exist
    :raise 403: if the current user isn't the author
    """
    post = g.service.read_post_by_id(g.user["id"],id)
    
    if post is None:
        abort(404, "Post id {0} doesn't exist.".format(id))

    if check_author and post["author_id"] != g.user["id"]:
        # TODO: this is no longer relevant
        abort(403)

    return post

In [ ]:
#export
@bp.route("/create", methods=("GET", "POST"))
@login_required
def create():
    """Create a new post for the current user."""
    if request.method == "POST":
        title = request.form["title"]
        body = request.form["body"]
        error = None

        if not title:
            error = "Title is required."

        if error is not None:
            flash(error)
        else:
            g.service.create_post(g.user["id"],title,body)
            return redirect(url_for("blog.index"))

    return render_template("blog/create.html")

In [ ]:
#export
@bp.route("/<int:id>/update", methods=("GET", "POST"))
@login_required
def update(id):
    """Update a post if the current user is the author."""
    post = get_post(id)

    if request.method == "POST":
        title = request.form["title"]
        body = request.form["body"]
        error = None

        if not title:
            error = "Title is required."

        if error is not None:
            flash(error)
        else:
            g.service.update_post_by_id(g.user["id"],id,['title','body'],[title,body])
            return redirect(url_for("blog.index"))

    return render_template("blog/update.html", post=post)

In [ ]:
#export
@bp.route("/<int:id>/delete", methods=("POST",))
@login_required
def delete(id):
    """Delete a post.
    Ensures that the post exists and that the logged in user is the
    author of the post.
    """
    get_post(id)
    g.service.delete_post_by_id(g.user["id"],id)
    return redirect(url_for("blog.index"))

In [ ]:
#export
@bp.route('/download')
@login_required
def download():
    directory,filename=g.service.prepare_posts_file_by_author_id(g.user["id"])
    if directory is None:
        username=g.user['username']
        abort(404, f'No data found for {username}')
    return send_from_directory(directory,filename,as_attachment=True)

In [ ]:
#export
@bp.route('/upload',methods=['POST'])
@login_required
def upload():
    if 'file' not in request.files:
        abort(400, 'No file part')
    file=request.files['file']
    if file.filename=='': 
        abort(400, 'No selected file')
    path=Path(tempfile.gettempdir())/f'entries-upload-{round(time.time()*1000)}'
    try:
        file.save(path)
        g.service.upload_posts_from_file(g.user["id"],path)
    finally:
        path.unlink()
    return "ok"

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 40a_service_db.ipynb.
Converted 40b_service_filesystem.ipynb.
Converted 50_web_app.ipynb.
Converted 50b_web_auth.ipynb.
Converted 50c_web_blog.ipynb.
Converted index.ipynb.
